In [42]:
from io import StringIO
import itertools
from typing import List

import pandas as pd
import requests

import fasttext.util
from joblib import dump, load
import pymorphy2
from sklearn import svm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, f1_score
from sklearn.neural_network import MLPClassifier

In [ ]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
df = pd.read_csv('data/train.csv')

df.head()

In [ ]:
other = ['привет как дела', 'привет', 'что делаешь', 'что ты умеешь',
          'ты тупой', 'какая погода', 'в радуге 7 цветов', 
          'какая-то рандомная фраза', 'пока', 'всего хорошего']
other_df = pd.DataFrame({'Пример текста': other, 'Класс': ['OTHER']*len(other)})
df = df.append(other_df)
df.reset_index(drop=True, inplace=True)

In [ ]:
df['label'] = pd.factorize(df['Класс'])[0]
df.rename({'Пример текста': 'text'}, axis=1, inplace=True)

df.label.value_counts()

In [ ]:
test_df = pd.read_csv('data/test.csv')

other = ['здравствуйте', 'чем занимаетесь', 'какие у тебя функции',
         'ты меня не понимаешь', 'до свидания']
other_df = pd.DataFrame({'Пример текста': other, 'Класс': ['OTHER']*len(other)})
test_df = test_df.append(other_df)
test_df.reset_index(drop=True, inplace=True)

test_df['label'] = pd.factorize(test_df['Класс'])[0]

X_test_text = test_df['Пример текста']
y_test = test_df.label.values

In [7]:
df.head()

,text,Класс,label
0,хочу в отпуск,VACATION-REQUEST,0
1,мне бы в отдохнуть,VACATION-REQUEST,0
2,как мне взять отпуск,VACATION-REQUEST,0
3,хочу отгул на следующей неделе,VACATION-REQUEST,0
4,хочу улететь в турцию,VACATION-REQUEST,0


In [8]:
labels_dict = {0: 'VACATION-REQUEST', 1: 'SALARY-REQUEST',
               2: 'SICK-LEAVE-REPORT', 3: 'OTHER'}

In [9]:
def pretty_print(data: List[List]):
    col_width = max(len(word) for row in data for word in row) + 2  # padding
    for row in data:
        print("".join(word.ljust(col_width) for word in row))

def print_errors(y_test, y_pred):
    wrong_idx = [idx for idx, (x, y) in enumerate(zip(y_pred, y_test)) if x!= y]

    data = [['Text', 'Classificator']]
    for idx in wrong_idx:
        data.append([X_test_text[idx], labels_dict[y_pred[idx]]])
    pretty_print(data)

## CountVec + LogisticRegression

In [10]:
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(df.text)
y_train = df.label.values

X_test = vectorizer.transform(X_test_text)

In [11]:
clf = LogisticRegression(random_state=42).fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.75      0.86         4
           1       1.00      1.00      1.00         4
           2       0.60      0.75      0.67         4
           3       0.80      0.80      0.80         5

    accuracy                           0.82        17
   macro avg       0.85      0.82      0.83        17
weighted avg       0.85      0.82      0.83        17



In [12]:
print_errors(y_test, y_pred)

Text                     Classificator            
Как мне получить оптуск  SICK-LEAVE-REPORT        
Бльничный нужен          OTHER                    
ты меня не понимаешь     SICK-LEAVE-REPORT        


## CountVec + SVM

In [13]:
# one-vs-rest
clf = svm.LinearSVC().fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.75      0.86         4
           1       1.00      1.00      1.00         4
           2       0.60      0.75      0.67         4
           3       0.80      0.80      0.80         5

    accuracy                           0.82        17
   macro avg       0.85      0.82      0.83        17
weighted avg       0.85      0.82      0.83        17



In [14]:
# one-vs-one
clf = svm.SVC(decision_function_shape='ovo').fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.75      0.75      0.75         4
           1       1.00      0.75      0.86         4
           2       0.40      0.50      0.44         4
           3       0.80      0.80      0.80         5

    accuracy                           0.71        17
   macro avg       0.74      0.70      0.71        17
weighted avg       0.74      0.71      0.72        17



## CountVec + MLPClassifier

In [15]:
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)

iterations = [50, 100, 150, 200]
solvers = ['lbfgs', 'sgd', 'adam']
activations = ['identity', 'logistic', 'tanh', 'relu']
params_tuples = list(itertools.product(iterations, solvers, activations))

scores = []
for it, solver, activation in params_tuples:
    clf = MLPClassifier(random_state=42, max_iter=it, 
                        solver=solver, activation=activation).fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    scores.append(f1_score(y_test, y_pred, average="weighted"))
print(f'Best num of iterations: {params_tuples[scores.index(max(scores))]}')

Best num of iterations: (50, 'lbfgs', 'logistic')


In [16]:
clf = MLPClassifier(random_state=42, max_iter=50, 
                    solver='lbfgs', activation='logistic').fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.75      0.86         4
           1       1.00      1.00      1.00         4
           2       0.75      0.75      0.75         4
           3       0.83      1.00      0.91         5

    accuracy                           0.88        17
   macro avg       0.90      0.88      0.88        17
weighted avg       0.89      0.88      0.88        17



## FastText + LogisticRegression

In [17]:
ft = fasttext.load_model('data/cc.ru.300.bin')

In [18]:
ft.get_dimension()

300

In [19]:
X_train = [ft.get_sentence_vector(sent) for sent in df.text]

In [20]:
clf = LogisticRegression(random_state=42).fit(X_train, y_train)
y_pred = clf.predict([ft.get_sentence_vector(sent) for sent in test_df['Пример текста']])

In [21]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.75      0.75      0.75         4
           1       0.57      1.00      0.73         4
           2       1.00      0.50      0.67         4
           3       1.00      0.80      0.89         5

    accuracy                           0.76        17
   macro avg       0.83      0.76      0.76        17
weighted avg       0.84      0.76      0.77        17



## FastText + MLPClassifier

In [22]:
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)

iterations = [50, 100, 150, 200]
solvers = ['lbfgs', 'sgd', 'adam']
activations = ['identity', 'logistic', 'tanh', 'relu']
params_tuples = list(itertools.product(iterations, solvers, activations))

scores = []
for it, solver, activation in params_tuples:
    clf = MLPClassifier(random_state=42, max_iter=it, 
                        solver=solver, activation=activation).fit(X_train, y_train)
    y_pred = clf.predict([ft.get_sentence_vector(sent) for sent in test_df['Пример текста']])
    scores.append(f1_score(y_test, y_pred, average="weighted"))
print(f'Best num of iterations: {params_tuples[scores.index(max(scores))]}')

Best num of iterations: (50, 'lbfgs', 'identity')


In [23]:
clf = MLPClassifier(random_state=42, max_iter=50, 
                    solver='lbfgs', activation='identity').fit(X_train, y_train)
y_pred = clf.predict([ft.get_sentence_vector(sent) for sent in test_df['Пример текста']])
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.75      0.86         4
           1       0.80      1.00      0.89         4
           2       0.80      1.00      0.89         4
           3       1.00      0.80      0.89         5

    accuracy                           0.88        17
   macro avg       0.90      0.89      0.88        17
weighted avg       0.91      0.88      0.88        17



## FastText+SVM

In [24]:
# one-vs-rest
clf = svm.LinearSVC().fit(X_train, y_train)
y_pred = clf.predict([ft.get_sentence_vector(sent) for sent in test_df['Пример текста']])
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.75      0.75      0.75         4
           1       0.57      1.00      0.73         4
           2       1.00      0.50      0.67         4
           3       1.00      0.80      0.89         5

    accuracy                           0.76        17
   macro avg       0.83      0.76      0.76        17
weighted avg       0.84      0.76      0.77        17



In [25]:
# one-vs-one
clf = svm.SVC(decision_function_shape='ovo').fit(X_train, y_train)
y_pred = clf.predict([ft.get_sentence_vector(sent) for sent in test_df['Пример текста']])
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.75      0.75      0.75         4
           1       1.00      1.00      1.00         4
           2       0.60      0.75      0.67         4
           3       1.00      0.80      0.89         5

    accuracy                           0.82        17
   macro avg       0.84      0.82      0.83        17
weighted avg       0.85      0.82      0.83        17



# Dataset Augmentation

## Word replacement

In [26]:
morph = pymorphy2.MorphAnalyzer()

In [27]:
def get_normal_form(word, morph):
    return morph.parse(word)[0].normal_form

In [28]:
def get_pos(word, morph):
    return morph.parse(word)[0].tag.POS

In [29]:
def transform_grammer_names(morh_grammer_name):
#     dict_morph_to_rusvectores = {'ADJF': 'ADJ', 'ADJS': 'ADJ'}
    dict_morph_to_rusvectores = {'NOUN':'NOUN'}
    return dict_morph_to_rusvectores.get(morh_grammer_name)

In [30]:
def get_synonyms(word):
    url = f'https://rusvectores.org/tayga_upos_skipgram_300_2_2019/{word}/api/csv/'
    r = requests.get(url)
    word_pos_df = pd.read_csv(StringIO(r.text), header=None, skiprows=2, sep='\t')
    return word_pos_df

In [31]:
def filter_synonyms(df, word, pos='ADJ'):
    df['pos'] = df[0].apply(lambda x: x.split('_')[1])
    df['word'] = df[0].apply(lambda x: x.split('_')[0])
    df = df[df.pos == pos]
    df.reset_index(drop=True, inplace=True)
    if df['word'][0] == word:
        return df['word'][1]
    return df['word'][0]

In [32]:
def replace_words(sent):
    sent = sent.split(' ')
    for idx, word in enumerate(sent):
        word = get_normal_form(word, morph)
        morph_pos = get_pos(word, morph)
        rus_vectores_pos = transform_grammer_names(morph_pos)
        if rus_vectores_pos:
            syn_df = get_synonyms(word + '_' + rus_vectores_pos)
            syn = filter_synonyms(syn_df, word, pos='NOUN')
            sent[idx] = syn
        else:
            continue
    return ' '.join(sent)

In [33]:
df['aug'] = df.text.apply(replace_words)

In [34]:
df.head()

,text,Класс,label,aug
0,хочу в отпуск,VACATION-REQUEST,0,хочу в отгул
1,мне бы в отдохнуть,VACATION-REQUEST,0,мне бы в отдохнуть
2,как мне взять отпуск,VACATION-REQUEST,0,как мне взять отгул
3,хочу отгул на следующей неделе,VACATION-REQUEST,0,хочу отпуск на следующей месяц
4,хочу улететь в турцию,VACATION-REQUEST,0,хочу улететь в россия


In [37]:
first_part = df[['text', 'label']].copy()
second_part = df[['aug', 'label']].copy()
second_part.rename({'aug': 'text'}, axis=1, inplace=True)
aug_df = first_part.append(second_part)

In [40]:
X_train = [ft.get_sentence_vector(sent) for sent in aug_df.text]
y_train = aug_df.label.values
clf = MLPClassifier(random_state=42, max_iter=50, 
                    solver='lbfgs', activation='identity').fit(X_train, y_train)
y_pred = clf.predict([ft.get_sentence_vector(sent) for sent in test_df['Пример текста']])
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.60      0.75      0.67         4
           1       0.75      0.75      0.75         4
           2       0.75      0.75      0.75         4
           3       1.00      0.80      0.89         5

    accuracy                           0.76        17
   macro avg       0.78      0.76      0.76        17
weighted avg       0.79      0.76      0.77        17



## Model saving

In [45]:
X_train = [ft.get_sentence_vector(sent) for sent in df.text]
y_train = df.label.values
clf = MLPClassifier(random_state=42, max_iter=50, 
                    solver='lbfgs', activation='identity').fit(X_train, y_train)
y_pred = clf.predict([ft.get_sentence_vector(sent) for sent in test_df['Пример текста']])
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.75      0.86         4
           1       0.80      1.00      0.89         4
           2       0.80      1.00      0.89         4
           3       1.00      0.80      0.89         5

    accuracy                           0.88        17
   macro avg       0.90      0.89      0.88        17
weighted avg       0.91      0.88      0.88        17



In [46]:
dump(clf, 'fasttext_mlpclassifier.joblib') 

['fasttext_mlpclassifier.joblib']

### Выводы
В рамках экспериментов попробовала BOW и FastText представления слов. Так как тестовый датасет очень маленький сложно объективно оценить на сколько хорошо работает модель. Лучше всех показали себя FastText + MLPClassifier, macro avg f1-score: 0.88.
Также, попробовала аугментировать тренировочный датасет, путем замены слов на синонимы. Для нахождения синонимов использовала rusvectores, для нахождения части речи использовала pymorphy2. Сначала пробовала заменять прилагательные и увидела, что замены получились не очень, тогда заменила существительные, что в принципе было приемлимло. Дальше попробовала применить лучшую полученную модель MLPClassifier с FastText эмбеддингами, результаты не улучшились. 

### Будущие эксперименты

#### Идеи с классификатором 
Так как датасет очень маленький, думаю, что тут хорошо будут работать и rule-based алгоритмы. 

#### Идеи с представлениями слов
Можно попробовать tf-idf. Если расширить датасет, то можно попробовать ELMo.

#### Идеи с аугментацией данных
Какую-то часть можно нагенерить руками. Также, можно поискать фразы в интернете, например, на сайтах изучения английского языка часто бывает наборы фраз, разделенные на интенты [пример](https://skyeng.ru/articles/100-poleznyh-razgovornyh-fraz-na-anglijskom). Когда данных будет больше, можно попробовать генеративные модели для генерации текста. 